In [1]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the pre-trained model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("bigcode/starcoder2-3b")
model = AutoModelForCausalLM.from_pretrained("bigcode/starcoder2-3b")

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# Prepare the model for LoRA training
model = get_peft_model(model, lora_config)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [9]:
import torch

# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")
    model.to(mps_device)
    print("Model moved to MPS device")

Model moved to MPS device


In [3]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("csv", data_files="code_files.csv", split="train[:90%]")

# Split the dataset into a training and testing set
dataset = dataset.train_test_split(test_size=0.1)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["Code"], max_length=512, truncation=True, padding="max_length")


# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     model.resize_token_embeddings(len(tokenizer))
tokenized_dataset = dataset.map(tokenize_function, batched=True)
# Define the training and evaluation datasets
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    evaluation_strategy="epoch",     # evaluation strategy to adopt during training
    per_device_train_batch_size=4,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    weight_decay=0.01,               # strength of weight decay
    use_cpu=True,                    # use CPU
     gradient_accumulation_steps=4, 
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,        # training dataset
    eval_dataset=eval_dataset,          # evaluation dataset
    data_collator=data_collator
)

trainer.train()





Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.386682152748108, 'eval_runtime': 7.223, 'eval_samples_per_second': 0.277, 'eval_steps_per_second': 0.138, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.384552001953125, 'eval_runtime': 6.9162, 'eval_samples_per_second': 0.289, 'eval_steps_per_second': 0.145, 'epoch': 2.0}
{'train_runtime': 434.6052, 'train_samples_per_second': 0.083, 'train_steps_per_second': 0.007, 'train_loss': 1.4669836362202961, 'epoch': 2.0}


TrainOutput(global_step=3, training_loss=1.4669836362202961, metrics={'train_runtime': 434.6052, 'train_samples_per_second': 0.083, 'train_steps_per_second': 0.007, 'train_loss': 1.4669836362202961, 'epoch': 2.0})

In [4]:
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/18.2M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/joshcarp/results/commit/651955b15e7bb28837e80c044651ba53623fec3f', commit_message='End of training', commit_description='', oid='651955b15e7bb28837e80c044651ba53623fec3f', pr_url=None, pr_revision=None, pr_num=None)

In [14]:
import torch
mps_device = torch.device("cpu")
model.to(mps_device)
def generate_text(prompt, max_length=100, num_return_sequences=1, device="cpu"):
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_beams=2,
        early_stopping=True,
    )

    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text

In [15]:
prompt = "Once upon a time, there was a curious programmer who wanted to"
generated_text = generate_text(prompt, max_length=200, num_return_sequences=1)
print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


KeyboardInterrupt: 

In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="joshcarp/results")

adapter_config.json:   0%|          | 0.00/673 [00:00<?, ?B/s]

In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

generated_text = pipe("Finish the code: func has_close_elements:bool numbers:[]num threshold:num", max_length=200, num_return_sequences=1)
print(generated_text["generated_text"])


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


KeyboardInterrupt: 